Pending work:
* check with different documents (health reports)
* play around with different embeddings
* choose a better model and measure performance( checkout johnsnowLabs)
* find a way to measure performance
* input all health reports to the model
* design the conversation between user and model
* few shot learning
* how to use the medical dataset
* check with different parameters
* UI implementation

You have to create a user token in huggingface. From profile, settings. Copy that to the executed block output and press enter

In [ ]:
import getpass
import os

# Enter your Hugging Face token here
hf_token = getpass.getpass("Enter your Hugging Face token: ")

# Set the HF_TOKEN environment variable
os.environ["HF_TOKEN"] = hf_token


In [ ]:
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-huggingface
!pip install llama-index ipywidgets
!pip install bitsandbytes
!pip install accelerate
!pip install llama-index
!pip install --upgrade streamlit
!pip install pyngrok

In [ ]:
!wget https://github.com/karan-nanonets/llamaindex-guide/raw/main/bcg-2022-annual-sustainability-report-apr-2023.pdf
!wget https://github.com/nandhishtr/DiseasePredictionLLM/blob/main/dataset_folder/health_report_%7B0%7D/health_report_%7B0%7D.txt

--2024-04-20 17:38:28--  https://github.com/karan-nanonets/llamaindex-guide/raw/main/bcg-2022-annual-sustainability-report-apr-2023.pdf
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/karan-nanonets/llamaindex-guide/main/bcg-2022-annual-sustainability-report-apr-2023.pdf [following]
--2024-04-20 17:38:28--  https://raw.githubusercontent.com/karan-nanonets/llamaindex-guide/main/bcg-2022-annual-sustainability-report-apr-2023.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19069924 (18M) [application/octet-stream]
Saving to: ‘bcg-2022-annual-sustainability-report-apr-2023.pdf’

bcg-2022-annual-

Base code

In [ ]:
import torch
import subprocess
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

# Model names (make sure you have access on HF)
LLAMA2_7B = "mistralai/Mistral-7B-v0.1"
selected_model = LLAMA2_7B

query_wrapper_prompt = PromptTemplate("{query_str}")

llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=1024,
    generate_kwargs={"temperature": 0.7, "do_sample": True},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=selected_model,
    model_name=selected_model,
    device_map="cuda:0",
    # change these settings below depending on your GPU
    model_kwargs={"torch_dtype": torch.float16, "load_in_4bit": True},
)

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

Settings.llm = llm
Settings.embed_model = embed_model

print('Disease prediction bot')

reader = SimpleDirectoryReader(
    input_files=["health_report_{0}.txt"]
)

text_documents = reader.load_data()
parser = SimpleNodeParser.from_defaults(chunk_size=1024, chunk_overlap=20)

text_nodes = parser.get_nodes_from_documents(text_documents)
index = VectorStoreIndex(text_nodes)

# # Clone the GitHub repository
# repo_url = "https://github.com/nandhishtr/DiseasePredictionLLM.git"
# repo_directory = "medical_reports"
# subprocess.run(["git", "clone", repo_url, repo_directory], check=True)
# # Read text files from the cloned repository
# text_documents = []
# for i in range(1):  # Assuming there are 100 health reports
#     report_folder = os.path.join(repo_directory, "dataset_folder", f"health_report_{i}")
#     report_file = os.path.join(report_folder, f"health_report_{i}.txt")
#     if os.path.exists(report_file):
#         with open(report_file, "r") as f:
#             text_documents.append(f.read())

# parser = SimpleNodeParser.from_defaults(chunk_size=1024, chunk_overlap=20)

# # Parse text documents into nodes
# text_nodes = parser.get_nodes_from_texts(text_documents)
# index = VectorStoreIndex(text_nodes)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
)
# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="simple_summarize",
)
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# while True:
prompt = input('User: ')
response = query_engine.query(prompt)
print('Bot:', response)


Previously tried code, along with streamlit

In [ ]:
import torch
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate
import streamlit as st
from pyngrok import ngrok

try:
    # Model names (make sure you have access on HF)
    LLAMA2_7B = "mistralai/Mistral-7B-v0.1"
    selected_model = LLAMA2_7B

    query_wrapper_prompt = PromptTemplate("{query_str}")

    llm = HuggingFaceLLM(
        context_window=2048,
        max_new_tokens=1024,
        generate_kwargs={"temperature": 0.0, "do_sample": False},
        query_wrapper_prompt=query_wrapper_prompt,
        tokenizer_name=selected_model,
        model_name=selected_model,
        device_map="cuda:0",
        # change these settings below depending on your GPU
        model_kwargs={"torch_dtype": torch.float16, "load_in_4bit": True},
    )

    st.title('Disease prediction bot')
    if 'messages' not in st.session_state:
        st.session_state.messages = []
    for message in st.session_state.messages:
        st.chat_message(message['role']).markdown(message['content'])

    prompt = st.chat_input('Pass your prompt here')
    if prompt:
        st.chat_message('user').markdown(prompt)
        st.session_state.messages.append({'role': 'user', 'content': prompt})
        response = llm(prompt)
        st.chat_message('assistant').markdown(response)
        st.session_state.messages.append({'role': 'assistant', 'content': response})

    # Setup a tunnel to the Streamlit port 8501
    public_url = ngrok.connect(port='8501')
    st.write("Public URL:", public_url)

except Exception as e:
    st.error(f"An error occurred: {e}")


In [ ]:
!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]